# Example 1

Example using a dataset collected by a single trifluoride boron detector manufactured by Hydroinnova, LLC.

Goal: Demonstrate the use of `CRNPy` to convert raw sensor data into surface soil water storage following common steps in the scientific literature.

Author: Andres Patrignani

In [3]:
from crnpy import crnpy
import pandas as pd
#from bokeh.plotting import figure, show, output_notebook
#output_notebook()


In [ ]:
# Parameter values
par = {'number_detectors':1,
       'count_time': 3600,
       'Pref':980, 
       'L':130, 
       'Aref':0,
       'N0':3486, 
       'Wlat':0.03,
       'Wsoc':0.01, 
       'bulk_density':1.33,
       'utc_offset':-6,
       'nmdb_station':'IRKT'}

variables = {'timestamp':[2],
             'temperature':[7],
             'humidity':[8],
             'pressure':[3],
             'counts_raw':[10],
             'counts_time':[11]}

In [ ]:
variables = {'timestamp':,
             'temperature':[7],
             'humidity':[8],
             'pressure':[3],
             'counts_raw':[10],
             'counts_time':[11]}

In [ ]:
cols_counts = ['counts_1','counts_2','counts_3']
df = (df.pipe(crnpy.format_dates,format='%m/%d/%Y %H:%M') \
       .pipe(crnpy.fill_counts,cols_counts,cols_time=[]) \
       .pipe(crnpy.compute_total_raw_counts, cols_counts))

In [ ]:
df.plot('timestamp','counts_1')


In [ ]:
df = df.pipe(crnpy.is_outlier, cols_counts, store_outliers=True)

In [ ]:
df.plot('timestamp','counts_1')


In [ ]:
df['counts_1'].isna().any()


In [ ]:
pd.to_datetime('2020-04-10 09:00:00') in source['timestamp'].values

In [ ]:
for date in date_range:
    if date not in source['timestamp'].values:
        print(date)
        new_line = pd.DataFrame({"timestamp":date}, index=[-1])
        source = pd.concat([source,new_line])
                                 
source.sort_values(by='timestamp', inplace=True)
source.reset_index(drop=True, inplace=True)
print(source.iloc[450:470])

In [ ]:
source.sort_values(by='timestamp', inplace=True)
source.reset_index(drop=True, inplace=True)
print(source.iloc[450:470])

In [ ]:
df.pipe(pycrns.fill_counts_v2,[['counts_1','counts_2','counts_3']],['counts_time'])

In [ ]:
# Correct neutron counts
#data = pycrns.load_data(filename,skip_rows,variables) 
#data = pycrns.fill_counts(data,par)

date = pycrns(filename,skip_rows,variables,par)
#data = data.fill_counts(par)
data = pycrns.fill_missing_atm(data)
data = pycrns.compute_total_counts(data)
data = pycrns.get_incoming_neutron_flux(data,par)
data = pycrns.atm_correction(data,par)
data = pycrns.smooth_counts(data)
data = pycrns.counts_to_vwc(data,par)
data = pycrns.sensing_depth(data,par,dist=[5,50,150])
data = pycrns.storage(data,T=5)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['counts_total_raw'], color='green', legend_label='N raw')
f.line(data['timestamp'], data['counts_total_corrected'], color='black', legend_label='N corrected')
f.line(data['timestamp'], data['counts_total_filtered'], color='tomato', legend_label='N filtered')
f.yaxis.axis_label = 'Neutron counts (cph)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['temperature'])
f.line(data['timestamp'], data['humidity'])
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['pressure'])
f.yaxis.axis_label = 'Atmospheric pressure (mbar)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['vwc'])
f.yaxis.axis_label = 'Volumetric water content (cm^3/cm^3)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['sensing_depth_D86_5m'], color='tomato', legend_label='5 m')
f.line(data['timestamp'], data['sensing_depth_D86_150m'], color='blue', legend_label='150 m')
f.yaxis.axis_label = 'Sensing depth (cm)'
f.legend.location = 'bottom_left'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['storage_surface'])
f.yaxis.axis_label = 'Storage (cmm)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['storage_rootzone'])
f.yaxis.axis_label = 'Storage (cmm)'
show(f)